In [231]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm

In [2]:
market_cap = pd.read_excel('Group_G copy.xlsx', sheet_name=1)
price_index = pd.read_excel('Group_G copy.xlsx', sheet_name=2, index_col='Date')
revenue = pd.read_excel('Group_G copy.xlsx', sheet_name=3)
emission = pd.read_excel('Group_G copy.xlsx', sheet_name=4)

In [50]:
def assets_returns(assets, period, compounded=True):
    if not compounded:
        asset_return = (assets / assets.shift(periods=period)) - 1
    elif compounded:
        diff = assets / assets.shift(periods=period)
        asset_return = np.log(diff.astype('float'))
    return asset_return

In [54]:
simple_return = assets_returns(price_index, period=1, compounded=False)
simple_return

,CAMPBELL SOUP - TOT RETURN IND (~U$) (#T) - CAMPBELL SOUP - TOT RETURN IND (~U$) (#T),ZOETIS A - TOT RETURN IND (~U$) (#T) - ZOETIS A - TOT RETURN IND (~U$) (#T),APARTMENT INV.& MAN.'A' - TOT RETURN IND (~U$) (#T) - APARTMENT INV.& MAN.'A' - TOT RETURN IND (~U$) (#T),WILLIAMS - TOT RETURN IND (~U$) (#T) - WILLIAMS - TOT RETURN IND (~U$) (#T),TNSC.'A' SBVTG. - TOT RETURN IND (~U$) (#T) - TNSC.'A' SBVTG. - TOT RETURN IND (~U$) (#T),BREAD FINANCIAL HOLDINGS - TOT RETURN IND (~U$) (#T) - BREAD FINANCIAL HOLDINGS - TOT RETURN IND (~U$) (#T),NASDAQ - TOT RETURN IND (~U$) (#T) - NASDAQ - TOT RETURN IND (~U$) (#T),ENERGEN DEAD - DELIST.30/11/18 - TOT RETURN IND (~U$) (#T) - ENERGEN DEAD - DELIST.30/11/18 - TOT RETURN IND (~U$) (#T),XEROX HOLDINGS - TOT RETURN IND (~U$) (#T) - XEROX HOLDINGS - TOT RETURN IND (~U$) (#T),UNIVERSAL HEALTH SVS.'B' - TOT RETURN IND (~U$) (#T) - UNIVERSAL HEALTH SVS.'B' - TOT RETURN IND (~U$) (#T),...,WSP GLOBAL - TOT RETURN IND (~U$) (#T) - WSP GLOBAL - TOT RETURN IND (~U$) (#T),CHIMERA INVESTMENT - TOT RETURN IND (~U$) (#T) - CHIMERA INVESTMENT - TOT RETURN IND (~U$) (#T),SYNOVUS FINANCIAL - TOT RETURN IND (~U$) (#T) - SYNOVUS FINANCIAL - TOT RETURN IND (~U$) (#T),FORTIS - TOT RETURN IND (~U$) (#T) - FORTIS - TOT RETURN IND (~U$) (#T),FREEHOLD ROYALTIES - TOT RETURN IND (~U$) (#T) - FREEHOLD ROYALTIES - TOT RETURN IND (~U$) (#T),CHEESECAKE FACTORY - TOT RETURN IND (~U$) (#T) - CHEESECAKE FACTORY - TOT RETURN IND (~U$) (#T),ROYAL CARIBBEAN GROUP - TOT RETURN IND (~U$) (#T) - ROYAL CARIBBEAN GROUP - TOT RETURN IND (~U$) (#T),ENTERPRISE PRDS.PTNS.LP. - TOT RETURN IND (~U$) (#T) - ENTERPRISE PRDS.PTNS.LP. - TOT RETURN IND (~U$) (#T),BIRCHCLIFF ENERGY - TOT RETURN IND (~U$) (#T) - BIRCHCLIFF ENERGY - TOT RETURN IND (~U$) (#T),SUPERIOR PLUS - TOT RETURN IND (~U$) (#T) - SUPERIOR PLUS - TOT RETURN IND (~U$) (#T)
Date,,,,,,,,,,,,,,,,,,,,,
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-04,-0.019318,NaN,0.015101,-0.032066,0.030542,NaN,NaN,-0.003204,-0.002122,0.016873,...,NaN,NaN,-0.002606,-0.004760,0.016243,-0.043171,-0.047274,0.084056,NaN,0.013898
1999-01-05,0.026652,NaN,-0.001637,-0.010350,-0.020790,NaN,NaN,0.000000,0.019105,0.004731,...,NaN,NaN,0.020889,0.013321,0.009300,-0.004439,0.039058,0.038704,NaN,0.002721
1999-01-06,-0.009481,NaN,0.004955,0.031381,-0.002646,NaN,NaN,-0.003216,0.019268,-0.001175,...,NaN,NaN,0.020459,0.017944,0.018716,-0.008832,0.051234,0.052293,NaN,0.014192
1999-01-07,-0.013731,NaN,-0.023092,0.000000,-0.005047,NaN,NaN,-0.022581,-0.004591,-0.028336,...,NaN,NaN,-0.047619,0.021613,0.042114,-0.037937,-0.022692,-0.007082,NaN,-0.001749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-12-27,0.004714,-0.003159,-0.027325,0.002701,0.006491,-0.010185,-0.004892,NaN,0.006185,0.003552,...,0.006489,0.016272,0.006834,0.006489,0.006490,-0.012793,-0.017874,0.005845,0.006487,0.006487
2022-12-28,-0.008863,-0.010126,-0.015446,-0.020653,-0.011865,-0.034947,-0.004595,NaN,-0.022539,-0.013380,...,-0.013214,-0.067686,-0.005703,-0.015304,-0.034375,-0.003477,-0.018401,-0.013700,-0.051007,-0.004856


In [55]:
log_return = assets_returns(price_index, period=1, compounded=True)
log_return

/Users/victoriawong/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


,CAMPBELL SOUP - TOT RETURN IND (~U$) (#T) - CAMPBELL SOUP - TOT RETURN IND (~U$) (#T),ZOETIS A - TOT RETURN IND (~U$) (#T) - ZOETIS A - TOT RETURN IND (~U$) (#T),APARTMENT INV.& MAN.'A' - TOT RETURN IND (~U$) (#T) - APARTMENT INV.& MAN.'A' - TOT RETURN IND (~U$) (#T),WILLIAMS - TOT RETURN IND (~U$) (#T) - WILLIAMS - TOT RETURN IND (~U$) (#T),TNSC.'A' SBVTG. - TOT RETURN IND (~U$) (#T) - TNSC.'A' SBVTG. - TOT RETURN IND (~U$) (#T),BREAD FINANCIAL HOLDINGS - TOT RETURN IND (~U$) (#T) - BREAD FINANCIAL HOLDINGS - TOT RETURN IND (~U$) (#T),NASDAQ - TOT RETURN IND (~U$) (#T) - NASDAQ - TOT RETURN IND (~U$) (#T),ENERGEN DEAD - DELIST.30/11/18 - TOT RETURN IND (~U$) (#T) - ENERGEN DEAD - DELIST.30/11/18 - TOT RETURN IND (~U$) (#T),XEROX HOLDINGS - TOT RETURN IND (~U$) (#T) - XEROX HOLDINGS - TOT RETURN IND (~U$) (#T),UNIVERSAL HEALTH SVS.'B' - TOT RETURN IND (~U$) (#T) - UNIVERSAL HEALTH SVS.'B' - TOT RETURN IND (~U$) (#T),...,WSP GLOBAL - TOT RETURN IND (~U$) (#T) - WSP GLOBAL - TOT RETURN IND (~U$) (#T),CHIMERA INVESTMENT - TOT RETURN IND (~U$) (#T) - CHIMERA INVESTMENT - TOT RETURN IND (~U$) (#T),SYNOVUS FINANCIAL - TOT RETURN IND (~U$) (#T) - SYNOVUS FINANCIAL - TOT RETURN IND (~U$) (#T),FORTIS - TOT RETURN IND (~U$) (#T) - FORTIS - TOT RETURN IND (~U$) (#T),FREEHOLD ROYALTIES - TOT RETURN IND (~U$) (#T) - FREEHOLD ROYALTIES - TOT RETURN IND (~U$) (#T),CHEESECAKE FACTORY - TOT RETURN IND (~U$) (#T) - CHEESECAKE FACTORY - TOT RETURN IND (~U$) (#T),ROYAL CARIBBEAN GROUP - TOT RETURN IND (~U$) (#T) - ROYAL CARIBBEAN GROUP - TOT RETURN IND (~U$) (#T),ENTERPRISE PRDS.PTNS.LP. - TOT RETURN IND (~U$) (#T) - ENTERPRISE PRDS.PTNS.LP. - TOT RETURN IND (~U$) (#T),BIRCHCLIFF ENERGY - TOT RETURN IND (~U$) (#T) - BIRCHCLIFF ENERGY - TOT RETURN IND (~U$) (#T),SUPERIOR PLUS - TOT RETURN IND (~U$) (#T) - SUPERIOR PLUS - TOT RETURN IND (~U$) (#T)
Date,,,,,,,,,,,,,,,,,,,,,
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-04,-0.019507,NaN,0.014988,-0.032592,0.030085,NaN,NaN,-0.003209,-0.002124,0.016733,...,NaN,NaN,-0.002609,-0.004772,0.016113,-0.044131,-0.048428,0.080710,NaN,0.013802
1999-01-05,0.026303,NaN,-0.001638,-0.010404,-0.021009,NaN,NaN,0.000000,0.018925,0.004720,...,NaN,NaN,0.020673,0.013233,0.009257,-0.004449,0.038315,0.037974,NaN,0.002717
1999-01-06,-0.009527,NaN,0.004943,0.030899,-0.002649,NaN,NaN,-0.003222,0.019085,-0.001176,...,NaN,NaN,0.020253,0.017785,0.018543,-0.008872,0.049965,0.050971,NaN,0.014092
1999-01-07,-0.013826,NaN,-0.023363,0.000000,-0.005060,NaN,NaN,-0.022839,-0.004602,-0.028745,...,NaN,NaN,-0.048790,0.021383,0.041252,-0.038676,-0.022953,-0.007107,NaN,-0.001751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-12-27,0.004703,-0.003164,-0.027706,0.002697,0.006470,-0.010237,-0.004904,NaN,0.006166,0.003546,...,0.006468,0.016141,0.006811,0.006468,0.006469,-0.012875,-0.018035,0.005828,0.006466,0.006466
2022-12-28,-0.008902,-0.010177,-0.015566,-0.020870,-0.011936,-0.035572,-0.004606,NaN,-0.022797,-0.013470,...,-0.013302,-0.070085,-0.005719,-0.015422,-0.034980,-0.003483,-0.018572,-0.013795,-0.052354,-0.004868


In [160]:
def get_moment(assetReturn, moment):
    if moment == 1:
        return assetReturn.mean()
    elif moment == 2:
        return assetReturn.std()
    elif moment == 3:
        return assetReturn.skew()
    elif moment == 4:
        nu = (pow(assetReturn, moment)).mean() - pow(assetReturn.mean(), moment)
        den = pow(assetReturn.std(), moment)
        res = nu / den
        return res

In [161]:
simple_mean = get_moment(simple_return, 1)
simple_std = get_moment(simple_return, 2)
simple_skew = get_moment(simple_return, 3)
simple_kurt = get_moment(simple_return, 4)

In [162]:
def jarque_bera(series, skewness, kurtosis):
    t = np.shape(series)[0]
    tstat = t * (pow(skewness, 2)/6 + pow((kurtosis-3), 2)/24)
    return tstat

In [191]:
jb_res = jarque_bera(simple_return, simple_skew, simple_kurt)
type(jb_res)

pandas.core.series.Series

In [204]:
def jb_test(tstat, alpha, dof):
    critical_val = stats.chi2.ppf(1-alpha, df=dof)
    out = {}
    for k, v in tstat.iteritems():
        if v >= critical_val:
            # print(f'Reject H0: Return is not Gaussian')
            out[k] = 'Reject H0: Return is not Gaussian'
    return pd.Series(out)

In [205]:
des = jb_test(jb_res, 0.05, 2)
des

CAMPBELL SOUP - TOT RETURN IND (~U$) (#T) - CAMPBELL SOUP - TOT RETURN IND (~U$) (#T)                          Reject H0: Return is not Gaussian
ZOETIS A - TOT RETURN IND (~U$) (#T) - ZOETIS A - TOT RETURN IND (~U$) (#T)                                    Reject H0: Return is not Gaussian
APARTMENT INV.& MAN.'A' - TOT RETURN IND (~U$) (#T) - APARTMENT INV.& MAN.'A' - TOT RETURN IND (~U$) (#T)      Reject H0: Return is not Gaussian
WILLIAMS - TOT RETURN IND (~U$) (#T) - WILLIAMS - TOT RETURN IND (~U$) (#T)                                    Reject H0: Return is not Gaussian
TNSC.'A' SBVTG. - TOT RETURN IND (~U$) (#T) - TNSC.'A' SBVTG. - TOT RETURN IND (~U$) (#T)                      Reject H0: Return is not Gaussian
                                                                                                                             ...                
CHEESECAKE FACTORY - TOT RETURN IND (~U$) (#T) - CHEESECAKE FACTORY - TOT RETURN IND (~U$) (#T)                Reject H0: Return i

In [269]:
def get_acf(ds: pd.DataFrame, nlags=10) -> pd.DataFrame:
    out = {}
    for k in ds.keys():
        res = sm.tsa.acf(ds[k], nlags=nlags, missing='conservative')
        out[k] = res[1:]
    return pd.DataFrame(out)

In [271]:
acf = get_acf(simple_return, nlags=10)
acf

/Users/victoriawong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:463: RuntimeWarning: invalid value encountered in double_scalars
  xo = x - x.sum() / notmask_int.sum()
/Users/victoriawong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:518: RuntimeWarning: invalid value encountered in true_divide
  acov = np.fft.ifft(Frf * np.conjugate(Frf))[:nobs] / d[nobs - 1 :]
/Users/victoriawong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:463: RuntimeWarning: invalid value encountered in subtract
  xo = x - x.sum() / notmask_int.sum()


,CAMPBELL SOUP - TOT RETURN IND (~U$) (#T) - CAMPBELL SOUP - TOT RETURN IND (~U$) (#T),ZOETIS A - TOT RETURN IND (~U$) (#T) - ZOETIS A - TOT RETURN IND (~U$) (#T),APARTMENT INV.& MAN.'A' - TOT RETURN IND (~U$) (#T) - APARTMENT INV.& MAN.'A' - TOT RETURN IND (~U$) (#T),WILLIAMS - TOT RETURN IND (~U$) (#T) - WILLIAMS - TOT RETURN IND (~U$) (#T),TNSC.'A' SBVTG. - TOT RETURN IND (~U$) (#T) - TNSC.'A' SBVTG. - TOT RETURN IND (~U$) (#T),BREAD FINANCIAL HOLDINGS - TOT RETURN IND (~U$) (#T) - BREAD FINANCIAL HOLDINGS - TOT RETURN IND (~U$) (#T),NASDAQ - TOT RETURN IND (~U$) (#T) - NASDAQ - TOT RETURN IND (~U$) (#T),ENERGEN DEAD - DELIST.30/11/18 - TOT RETURN IND (~U$) (#T) - ENERGEN DEAD - DELIST.30/11/18 - TOT RETURN IND (~U$) (#T),XEROX HOLDINGS - TOT RETURN IND (~U$) (#T) - XEROX HOLDINGS - TOT RETURN IND (~U$) (#T),UNIVERSAL HEALTH SVS.'B' - TOT RETURN IND (~U$) (#T) - UNIVERSAL HEALTH SVS.'B' - TOT RETURN IND (~U$) (#T),...,WSP GLOBAL - TOT RETURN IND (~U$) (#T) - WSP GLOBAL - TOT RETURN IND (~U$) (#T),CHIMERA INVESTMENT - TOT RETURN IND (~U$) (#T) - CHIMERA INVESTMENT - TOT RETURN IND (~U$) (#T),SYNOVUS FINANCIAL - TOT RETURN IND (~U$) (#T) - SYNOVUS FINANCIAL - TOT RETURN IND (~U$) (#T),FORTIS - TOT RETURN IND (~U$) (#T) - FORTIS - TOT RETURN IND (~U$) (#T),FREEHOLD ROYALTIES - TOT RETURN IND (~U$) (#T) - FREEHOLD ROYALTIES - TOT RETURN IND (~U$) (#T),CHEESECAKE FACTORY - TOT RETURN IND (~U$) (#T) - CHEESECAKE FACTORY - TOT RETURN IND (~U$) (#T),ROYAL CARIBBEAN GROUP - TOT RETURN IND (~U$) (#T) - ROYAL CARIBBEAN GROUP - TOT RETURN IND (~U$) (#T),ENTERPRISE PRDS.PTNS.LP. - TOT RETURN IND (~U$) (#T) - ENTERPRISE PRDS.PTNS.LP. - TOT RETURN IND (~U$) (#T),BIRCHCLIFF ENERGY - TOT RETURN IND (~U$) (#T) - BIRCHCLIFF ENERGY - TOT RETURN IND (~U$) (#T),SUPERIOR PLUS - TOT RETURN IND (~U$) (#T) - SUPERIOR PLUS - TOT RETURN IND (~U$) (#T)
0,-0.034562,-0.078388,-0.104573,0.087802,-0.050844,0.062442,0.015684,-0.051927,-0.025333,-0.030117,...,0.029233,-0.086876,-0.082349,-0.029875,0.020075,0.005523,0.054312,0.039466,0.004989,0.049940
1,-0.028517,0.017861,0.043051,0.009481,-0.029134,0.008906,0.006765,-0.009212,0.010935,0.016330,...,-0.049816,-0.068209,-0.004398,-0.028132,-0.029234,-0.065797,0.002548,-0.048153,-0.001636,0.021926
2,-0.018136,-0.020288,-0.010478,0.039892,-0.012398,-0.061882,-0.046791,-0.013752,-0.003656,-0.016747,...,0.007818,-0.002895,-0.037566,-0.013395,0.017550,-0.045370,-0.035153,-0.014788,-0.017376,-0.005370
3,-0.020476,-0.011505,-0.000756,-0.084129,0.001323,-0.021815,0.006251,0.005226,-0.001018,-0.001082,...,0.044230,-0.037536,-0.001257,-0.014111,0.034307,-0.024179,0.014568,0.000038,0.006589,0.014209
4,-0.032980,0.003695,-0.055793,-0.136199,0.033284,-0.019336,-0.024357,-0.032487,-0.043195,-0.000565,...,0.004930,-0.005719,-0.038955,-0.034877,-0.036058,-0.002800,-0.024173,-0.027974,-0.002358,-0.017662
5,0.001236,-0.057794,-0.011137,-0.070469,-0.037826,-0.070572,-0.028563,0.008664,0.033331,-0.032776,...,-0.062512,-0.042196,-0.027794,-0.065743,-0.028914,-0.019000,-0.024231,-0.050704,0.010451,-0.062888
6,-0.007997,0.023837,-0.003721,-0.051685,-0.007549,0.027214,-0.003891,-0.013735,0.022737,0.035089,...,-0.014225,-0.000570,0.003930,0.034012,0.039873,0.030246,0.008431,-0.005401,-0.007475,0.031116
7,0.031145,-0.061895,0.020125,-0.029795,0.020962,-0.029483,0.008940,0.005973,-0.038628,-0.005894,...,-0.008795,0.061220,0.002118,-0.004333,0.012422,-0.010236,0.009586,-0.041267,0.000488,-0.006843
8,0.000537,0.054412,0.003167,0.032794,-0.014886,0.032387,0.013361,0.017520,0.015944,0.022858,...,0.011540,0.004226,0.002778,-0.001509,-0.010390,0.034584,0.015398,0.006036,0.001246,0.021012
9,-0.001959,-0.036152,0.038233,0.033410,0.010553,0.013443,0.000979,0.007600,-0.014625,-0.010483,...,-0.015694,0.031993,0.010030,-0.019178,-0.019831,-0.034093,0.029313,0.017671,-0.009834,-0.016434
